# Tutorial 1: calibration

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import os

In [2]:
url = 'https://raw.githubusercontent.com/oguerrer/ppi_app/main/code/ppi.py'
r = requests.get(url)
with open('ppi.py', 'w') as f:
    f.write(r.text)
import ppi

url = 'https://raw.githubusercontent.com/oguerrer/ppi_app/main/code/calibrator.py'
r = requests.get(url)
with open('calibrator.py', 'w') as f:
    f.write(r.text)
import calibrator

### Download datasets

In [3]:
file_names = ['expenditure_matrix_imperfect', 'expenditure_matrix_perfect', 'expenditure_mean',
        'expenditure_serie', 'indicators_properties', 'indicators_series', 'network']

for file_name in file_names:
    url = 'https://raw.githubusercontent.com/oguerrer/ppi_app/main/data/benchmark/'+file_name+'.csv'
    r = requests.get(url)
    with open(os.path.split(url)[1], 'wb') as f, \
            requests.get(url, stream=True) as r:
        for line in r.iter_lines():
            f.write(line+'\n'.encode())

### Calibration with indicator time series only

In [4]:
# Load indicators
series = np.loadtxt('indicators_series.csv', delimiter=',', dtype=float)
# Get initial values
I0 = series[:,0]
# Get final values
IF = series[:,-1]
# Get success rates
success_rates = np.sum(series[:,1::] < series[:,0:-1], axis=1)/(series.shape[1]-1)

In [5]:
parameters = calibrator.calibrate(I0, IF, success_rates, parallel_processes=20)
df_indicators_series = pd.DataFrame(parameters[1::,:], columns=parameters[0])